In [1]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [3]:
fake_B = torch.zeros((3,4,5))
fake_B[0][0][1] = 1
fake_B[0][1][0] = 1
fake_B[0][2][3] = 1
fake_B

tensor([[[0., 1., 0., 0., 0.],
         [1., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

In [4]:

a  = (fake_B[:, :,0] != 1).long()

In [5]:
model = torch.load('./model/D_A.pt')

In [6]:
y,y_attn = en(['a certain number of workers in certain operating sites, on which this type of directive a due due to practical circumstances circumstances, only a difficult difficult to to use.....', 'I myself from Schottland, a region with one of the largest fishing fishing fleets in the EU EU, will again again on the problem of fisheries and the definition of the definition of the working time..................'])

In [7]:
label,label_attn = en(['There would remain though some workers in some working places where practical issues do not readily lend themselves to the implementation of this type of directive.', 'Coming from Scotland, with one of the largest EU fishing fleets, I would like once again to focus on the problem of fisheries and the defining of working time.'])

In [8]:
model(y.cuda(),y_attn.cuda())

distr.shape torch.Size([2, 50, 512])
distr.shape torch.Size([2, 512])
ret.shape torch.Size([2, 1])


tensor([[0.1544],
        [0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)

In [9]:
model(label.cuda(),label_attn.cuda())

distr.shape torch.Size([2, 37, 512])
distr.shape torch.Size([2, 512])
ret.shape torch.Size([2, 1])


tensor([[0.0700],
        [0.0000]], device='cuda:0', grad_fn=<ReluBackward0>)

In [26]:
#real sentences
x = [[   37,   351,    13,     8,   657,  1267,    24,    62,     6,    38,
          8160,  8675,    13,     8,  1611, 15375,     7,     6,   225,  2173,
           981,  1131,  1106,  6833,    11, 13567,    16,     8,  1942,    13,
           175,  2731,    11,     8,  6572,    53,    24,     3,    35,  5756,
             7,     6,    11,    28,    69, 12123,     7,    11,  7548,     6,
            62,   241,    12,  1984,   125,    54,    36,   261,    38,    46,
         11169,    13,     8,   167, 24367,     3, 30786,  1127,     5,     1],
        [    3, 23531,     6,    66,   396,   557,     6,  1450,  4505,     7,
            33,   271,  1869,    30,  1195,     3,  2544,   772,    44,     8,
          1849,    13,     8,  2486,    33,   914,    59,   780,   964,     5,
             1,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]
             

x_ = torch.Tensor(x).long().cuda()
x_ = x_[:,:]
x_attn = (x_>0.5).long()
x_attn_ =x_attn
print(x_.shape)
print(x_attn.shape)


torch.Size([2, 70])
torch.Size([2, 70])


In [28]:

x_emb = model.embedding(x_)#
print('x_emb.shape',x_emb.shape)
distr = model.encoder(inputs_embeds=x_emb).last_hidden_state#(bs,sentence length,512)
print('distr.shape',distr.shape)
x_attn= x_attn.unsqueeze(-1)
print('x_attn.shape',x_attn.shape)
distr = torch.mul(distr,x_attn)#previously
print('mutil',torch.sum(distr,1).shape)
print('count',torch.sum(x_attn,1).shape)
distr = torch.sum(distr,1)/torch.sum(x_attn,1)#(bs,512)
print('mea',distr.shape)

ret =  model.classifier(distr)#(bs,1)
print(ret)
ret = model.relu(ret)#(bs,1)
print(ret)



a_pred_dis = model(x_,x_attn_)
a_pred_dis

x_emb.shape torch.Size([2, 70, 512])
distr.shape torch.Size([2, 70, 512])
x_attn.shape torch.Size([2, 70, 1])
mutil tensor([[[ 0.6085,  0.0000, -0.0000,  ..., -0.1359,  0.0513, -0.2267],
         [ 0.5553,  0.1303, -0.0000,  ...,  0.0871,  0.0641, -0.0583],
         [-0.0642,  0.0000, -0.0356,  ..., -0.0681,  0.1548, -0.1681],
         ...,
         [ 0.0430,  0.0789, -0.0459,  ...,  0.1795,  0.0000, -0.1211],
         [-0.1309,  0.3895, -0.1361,  ..., -0.0000, -0.0494, -0.1605],
         [ 0.0418,  0.0000, -0.0425,  ..., -0.0185, -0.0090, -0.0287]],

        [[ 0.1594,  0.2739, -0.0671,  ...,  0.0419,  0.0021, -0.3445],
         [ 0.7014,  0.3530,  0.2514,  ..., -0.1553, -0.0000, -0.0291],
         [ 0.0173,  0.2503,  0.0305,  ..., -0.3229, -0.1662, -0.1849],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [-0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000, -0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],


tensor([[0.0139],
        [0.3691]], device='cuda:0', grad_fn=<ReluBackward0>)

## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier